
# Tema 8 – Predicción de compradores recurrentes (Online Retail – *Auto*)
**Curso:** CC3084 – Data Science (Sem II 2025)  
**Equipo:** _Mauricio Lemus - 22461, Alexis Mesias - 22562, Hugo Rivas - 22500_  
**Repositorio GitHub:** _[https://github.com/Riv2oo4/Proyecto2_DataScience.git]_  


## 0) Preparación del entorno 

In [ ]:

import os, sys, io, warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt
pd.set_option("display.max_columns", 120)
warnings.filterwarnings("ignore")

def in_colab():
    try:
        import google.colab  
        return True
    except Exception:
        return False

def kaggle_token_present():
    return os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")) or os.path.exists("./kaggle.json")

print("Entorno Colab:", in_colab())
print("kaggle.json presente:", kaggle_token_present())


Entorno Colab: False
kaggle.json presente: False


## 1) Descargar dataset Online Retail 

In [ ]:

DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)
DATA_PATH = os.path.join(DATA_DIR, "Online Retail.xlsx")

def try_kaggle_download():
    try:
        import subprocess, shutil, json, os
        if os.path.exists("./kaggle.json"):
            os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
            shutil.copyfile("./kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))
            os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

        try:
            import kaggle  
        except Exception:
            subprocess.run([sys.executable, "-m", "pip", "install", "kaggle", "--quiet"], check=True)

        # Descargar y descomprimir
        zip_path = os.path.join(DATA_DIR, "onlineretail.zip")
        subprocess.run(["kaggle", "datasets", "download", "-d", "vijayuv/onlineretail", "-p", DATA_DIR, "--force"], check=True)
        subprocess.run(["unzip", "-o", zip_path, "-d", DATA_DIR], check=True)
        print("Descarga Kaggle completa.")
        return True
    except Exception as e:
        print("Kaggle no disponible o fallo descarga:", e)
        return False

if not os.path.exists(DATA_PATH) and kaggle_token_present():
    _ = try_kaggle_download()

print("DATA_PATH existe:", os.path.exists(DATA_PATH), "->", DATA_PATH if os.path.exists(DATA_PATH) else "No encontrado")


DATA_PATH existe: False -> No encontrado


## 2) Subir archivo manualmente 

In [3]:

if not os.path.exists(DATA_PATH):
    if in_colab():
        try:
            from google.colab import files  # type: ignore
            print("Sube el archivo Excel del dataset (e.g., 'Online Retail.xlsx').")
            uploaded = files.upload()
            # Guardar con nombre estándar si el usuario sube algo
            for name, data in uploaded.items():
                with open(DATA_PATH, "wb") as f:
                    f.write(data)
            print("Archivo guardado en:", DATA_PATH)
        except Exception as e:
            print("Subida manual no disponible:", e)
    else:
        print("No estás en Colab. Puedes colocar el Excel en ./data con nombre 'Online Retail.xlsx'.")


No estás en Colab. Puedes colocar el Excel en ./data con nombre 'Online Retail.xlsx'.


## 3) Cargar datos (o crear sintético si no hay archivo)

In [ ]:

import pandas as pd, numpy as np

df = None
if os.path.exists(DATA_PATH):
    try:
        df = pd.read_excel(DATA_PATH)
        print("Cargado Excel:", DATA_PATH, "->", df.shape)
    except Exception as e:
        print("Error al leer Excel, se usará dataset sintético:", e)

if df is None:
    # ---- Synthetic fallback (guaranteed to run) ----
    print("Usando dataset sintético para ejecutar el flujo completo.")
    rng = np.random.default_rng(42)
    n_customers = 1500
    start = pd.Timestamp("2024-01-01")
    end = pd.Timestamp("2025-08-31")
    days = (end - start).days + 1

    customers = pd.DataFrame({
        "CustomerID": np.arange(1, n_customers+1),
        "Channel": rng.choice(["web","app","marketplace"], size=n_customers, p=[0.5,0.4,0.1]),
        "Device": rng.choice(["mobile","desktop","tablet"], size=n_customers, p=[0.6,0.35,0.05]),
    })

    orders_per_cust = rng.poisson(lam=1.6, size=n_customers) + (rng.random(n_customers) < 0.25)
    orders_per_cust = orders_per_cust.astype(int)
    orders_per_cust[orders_per_cust<1] = 1

    rows = []
    order_id = 1
    for cid, k in zip(customers["CustomerID"].values, orders_per_cust):
        for _ in range(k):
            d = start + pd.to_timedelta(rng.integers(0, days), unit="D")
            qty = int(max(1, rng.poisson(2)))
            price = float(max(1.0, np.round(rng.gamma(shape=2.2, scale=5.0), 2)))
            rows.append([str(order_id), str(cid), d, qty, price])
            order_id += 1

    df = pd.DataFrame(rows, columns=["InvoiceNo","CustomerID","InvoiceDate","Quantity","UnitPrice"])
    print("Sintético generado:", df.shape)

# Renombrar columnas a snake_case y crear 'amount'
rename_map = {
    "InvoiceNo": "invoice_no",
    "StockCode": "stock_code",
    "Description": "description",
    "Quantity": "quantity",
    "InvoiceDate": "invoice_date",
    "UnitPrice": "unit_price",
    "CustomerID": "customer_id",
    "Country": "country",
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

df["invoice_date"] = pd.to_datetime(df["invoice_date"], errors="coerce")
df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce")
df["unit_price"] = pd.to_numeric(df["unit_price"], errors="coerce")
df["customer_id"] = pd.to_numeric(df["customer_id"], errors="coerce")
df["amount"] = df.get("quantity", 1) * df.get("unit_price", 1.0)

print("Preview:")
display(df.head(5))
print(df.dtypes)


## 4) Chequeos iniciales

In [ ]:

print("Nulos por columna:")
print(df.isna().sum())

if "invoice_date" in df.columns:
    print("\nRango de fechas:")
    print(df["invoice_date"].min(), "->", df["invoice_date"].max())

if "amount" in df.columns:
    print("\n'amount' describe:")
    print(df["amount"].describe())


## 5) Limpieza básica y agregación por orden

In [ ]:

data = df.copy()

# Filtrar notas de crédito si existen (InvoiceNo que empieza con 'C')
if "invoice_no" in data.columns:
    mask_credit = data["invoice_no"].astype(str).str.startswith("C", na=False)
    data = data[~mask_credit].copy()

# Filtrar cantidades y precios válidos
for col in ["quantity","unit_price","customer_id","invoice_date"]:
    if col in data.columns:
        pass
data = data[(data.get("quantity", 1) > 0) & (data.get("unit_price", 1.0) > 0)]
data = data.dropna(subset=["customer_id","invoice_date"])

# Definir order_id y agrupar
data["order_id"] = data.get("invoice_no", data.index.astype(str))
orders = (
    data.groupby(["order_id", "customer_id", "invoice_date"], as_index=False)
        .agg(amount=("amount","sum"))
)
print("Órdenes únicas:", len(orders))
display(orders.head(5))
